This notebook is associated with the paper "The relative class number one problem for function fields, III" by K.S. Kedlaya. It runs in SageMath (tested using version 9.4) and depends on Magma (tested using version
2.25-5).

In this notebook, we identify curves of genus 7 with a self-adjoint $g^2_6$ (i.e., one which squares to the canonical class) which are candidates for the curve $C$ in a purely geometric quadratic extension $F'/F$ of relative class number 1. Allow 5 minutes for completion.

In [1]:
load("preamble.sage")

Construct the set of $\mathbb{F}_2$-rational points of $\mathbf{P}(1:1:1:2)$.

In [2]:
F = GF(2)
P.<x0,x1,x2,y> = PolynomialRing(F, 4)

In [3]:
S = [vector(t) for t in ProjectiveSpace(F, 3)]
for v in S:
    v.set_immutable()
assert list(S[0]) == [0,0,0,1]

Construct pairs of cubic and quartic hypersurfaces in $\mathbf{P}(1:1:1:2)$ whose $\mathbb{F}_2$-rational points match a given 6-tuple or 7-tuple not including the singular point $[0:0:0:1]$.

In [4]:
gens1 = [a+b for a in [0, x0*y] for b in [x1*x2*(x1+x2), x1*(x1^2+x1*x2+x2^2), x1^3+x1*x2^2+x2^3]]
mons4 = [prod(P.gens()[i]^w[i] for i in range(4)) for w in WeightedIntegerVectors(4, [1,1,1,2])]

In [5]:
coords4 = {x: vector(mu(*x) for mu in mons4) for x in S}

In [6]:
def redundancy(gen1, P=P, F=F, mons4=mons4):
    tmp = [gen1*x for x in P.gens()[:3]]
    return [vector(F, (gen.coefficient(mu) for mu in mons4)) for gen in tmp]

In [7]:
l = []
V0 = VectorSpace(F, len(mons4))
curves = defaultdict(list)
for gen1 in gens1:
    pts = [x for x in S if gen1(*x) == 0]
    perp = Matrix([coords4[x] for x in pts])
    for s in [6, 7]:
        for pts1 in itertools.combinations([x for x in pts if x != S[0]], s):
            target = vector(F, (0 if x in pts1 else 1 for x in pts))
            for v2 in solve_right_iterator(perp, target, redundancy, gen1):
                gen2 = sum(v2[i]*mons4[i] for i in range(len(mons4)))
                curves[(s,)].append([gen1, gen2])

In [8]:
[(s, len(curves[s])) for s in curves]

[((7,), 140544), ((6,), 61952)]

Enforce the desired point counts over $\mathbb{F}_{2^i}$ for $i=2,3$ using commutative algebra.

In [9]:
def count_by_ideal(gens, n):
    J = P.ideal(gens + [y^(2^n) + y for y in P.gens()])
    return (J.vector_space_dimension() - 1) // (2^n-1)

In [10]:
for n in range(2, 4):
    tmp = set(t[:n] for t in targets7)
    tmp2 = [s for s in curves if len(s) == n-1]    
    for s in tmp2:
        for gens in curves[s]:
            i = count_by_ideal(gens, n)
            s1 = s + (i,)
            if s1 in tmp:
                curves[s1].append(gens)
        del curves[s]
    print([(s, len(curves[s])) for s in curves if len(s) == n])

[((7, 15), 25350), ((6, 18), 202)]
[((7, 15, 7), 102), ((6, 18, 12), 48)]


Close out this case.

In [11]:
I1 = P.ideal([x0,x1,x2,y])
CR = magma.CoxRing(P, [I1], [[1,1,1,2]], [])
proj = CR.ToricVariety()

In [12]:
closeout(curves, X=proj, genus=7)

Number of curves found: 0
Number of isomorphism classes found: 0
No covers found in this case!
Total time: 5 minutes
